In [1]:
import pandas as pd
import numpy as np
import os
import csv
import requests
from urllib import request, error
from PIL import Image
from io import BytesIO
from datetime import datetime
from tqdm import tqdm

In [2]:
girl = pd.read_csv('data/girl_art.csv')
girl.head(5)
boy = pd.read_csv('data/boy_art.csv')
boy.head(5)

,id,created_at,rating,score,sample_url,sample_width,sample_height,preview_url,tags
0,111,1264803408,s,6,//safebooru.org/images/1/0f06585c990985a740b89...,480,1200,//safebooru.org/thumbnails/1/thumbnail_0f06585...,boy
1,132,1264803422,s,5,//safebooru.org/images/1/4054c004887474c5bd809...,567,800,//safebooru.org/thumbnails/1/thumbnail_4054c00...,boy
2,283,1264803550,s,0,//safebooru.org/samples/1/sample_2d52076d1ec3f...,850,513,//safebooru.org/thumbnails/1/thumbnail_2d52076...,boy
3,310,1264803570,s,0,//safebooru.org/samples/1/sample_56ea37fa0d914...,850,531,//safebooru.org/thumbnails/1/thumbnail_56ea37f...,boy
4,377,1264803623,s,2,//safebooru.org/images/1/e0826090e906fb6abe5b0...,1680,1050,//safebooru.org/thumbnails/1/thumbnail_e082609...,boy


In [4]:
girl_data = girl.to_numpy()
print(type(girl_data))
boy_data = boy.to_numpy()
print(type(boy_data))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [5]:
#get new list of 32000 data
idx = np.random.choice(girl_data.shape[0], 32000, replace=False)
girl_sample = np.array(girl_data[idx,:])
print(len(girl_sample))
idx = np.random.choice(boy_data.shape[0], 32000, replace=False)
boy_sample = np.array(boy_data[idx,:])
print(len(boy_sample))

32000
32000


In [6]:
def make_dir(path):
    try:
        os.mkdir(path)
    except:
        print('Directory ', path, ' already exist')

In [7]:
IMAGE_DIR = 'image'
TRAINING_DIR = os.path.join(IMAGE_DIR, 'train')
VALIDATION_DIR = os.path.join(IMAGE_DIR, 'val')
make_dir(TRAINING_DIR)
make_dir(VALIDATION_DIR)

Directory  image\train  already exist
Directory  image\val  already exist


In [8]:
BOY_TRAIN = os.path.join(TRAINING_DIR, 'boy')
GIRL_TRAIN = os.path.join(TRAINING_DIR, 'girl')
BOY_VAL = os.path.join(VALIDATION_DIR, 'boy')
GIRL_VAL = os.path.join(VALIDATION_DIR, 'girl')
make_dir(BOY_TRAIN)
make_dir(GIRL_TRAIN)
make_dir(BOY_VAL)
make_dir(GIRL_VAL)

Directory  image\train\boy  already exist
Directory  image\train\girl  already exist
Directory  image\val\boy  already exist
Directory  image\val\girl  already exist


In [9]:
BOY_SAMPLE = 'sample/boy'
GIRL_SAMPLE = 'sample/girl'
boy_url_sample = np.array(boy_sample[:,-2])
girl_url_sample = np.array(girl_sample[:,-2])

In [10]:
pathinit = 'https:'
for idx in tqdm(range(len(boy_url_sample))):
    if pathinit not in boy_url_sample[idx]:
        boy_url_sample[idx] = pathinit + boy_url_sample[idx]
print(boy_url_sample)
for idx in tqdm(range(len(girl_url_sample))):
    if pathinit not in girl_url_sample[idx]:
        girl_url_sample[idx] = pathinit + girl_url_sample[idx]
print(girl_url_sample)

100%|████████████████████████████████████████████████████████████████████████| 32000/32000 [00:00<00:00, 532751.68it/s]

['https://safebooru.org/thumbnails/1668/thumbnail_93cf2b2735f290bc105f870213e4407751e51fe6.png'
 'https://safebooru.org/thumbnails/1827/thumbnail_f9221ef14b6cf1d9e6363476e63f1b5bb738fc5d.jpeg'
 'https://safebooru.org/thumbnails/2242/thumbnail_ecb05b09514e7c5a0b70173443abbbcf8d5de293.jpg'
 ...
 'https://safebooru.org/thumbnails/2908/thumbnail_988ce4563f62fb8835963be5de5e059847729b1e.png'
 'https://safebooru.org/thumbnails/1898/thumbnail_cc6d155788bbd331c46ee41c5e0fa8c8ebab1526.gif'
 'https://safebooru.org/thumbnails/2829/thumbnail_4d9e3ed7912fbf2848b3bdd94153d55d47ba51b6.jpg']
['https://safebooru.org/thumbnails/2975/thumbnail_01ddff18768ea541a30493ac4497cbe59fa4a8d5.jpg'
 'https://safebooru.org/thumbnails/2229/thumbnail_82c612179f1b9e59f96bdb49d40865d6206a60b7.jpg'
 'https://safebooru.org/thumbnails/2127/thumbnail_ac0fc4093c8c4a9fbfdb32a495b7c118bb7d641b.jpg'
 ...
 'https://safebooru.org/thumbnails/1547/thumbnail_5fdcb1c1ce5f4bb03e977a15f9d6fe0b72972042.jpg'
 'https://safebooru.org/thum

In [11]:
def download_image(key, url):
    out_dir = 'sample'
    filename = os.path.join(out_dir, '{}.jpg'.format(key))

    if os.path.exists(filename):
        print('Image {} already exists. Skipping download.'.format(filename))
        return

    try:
        response = request.urlopen(url)
        image_data = response.read()
    except:
        print('Warning: Could not download image {} from {}'.format(key, url))
        return

    try:
        pil_image = Image.open(BytesIO(image_data))
    except:
        print('Warning: Failed to parse image {}'.format(key))
        return

    try:
        pil_image_rgb = pil_image.convert('RGB')
    except:
        print('Warning: Failed to convert image {} to RGB'.format(key))
        return

    try:
        pil_image_rgb.save(filename, format='JPEG', quality=90)
    except:
        print('Warning: Failed to save image {}'.format(filename))
        return

In [ ]:
for idx in tqdm(range(len(boy_url_sample))):
    url = boy_url_sample[idx]
    download_image(idx, url)
for idx in tqdm(range(len(girl_url_sample))):
    url = girl_url_sample[idx]
    download_image(idx, url)

  0%|                                                                             | 1/32000 [00:06<57:55:26,  6.52s/it]

  0%|                                                                             | 2/32000 [00:07<44:28:12,  5.00s/it]

  0%|                                                                             | 4/32000 [00:11<29:04:48,  3.27s/it]

  0%|                                                                             | 6/32000 [00:15<23:43:37,  2.67s/it]

  0%|                                                                             | 7/32000 [00:16<19:26:31,  2.19s/it]

...

  0%|                                                                            | 27/32000 [00:53<13:41:17,  1.54s/it]

  0%|                                                                            | 28/32000 [00:54<12:23:47,  1.40s/it]

  0%|                                                                            | 29/32000 [00:56<13:06:26,  1.48s/it]

  0%|                                                                            | 34/32000 [01:02<12:29:27,  1.41s/it]

  0%|                                                                            | 36/32000 [01:06<13:46:13,  1.55s/it]

In [62]:
training_size = 90 #in percent
arrint = np.arange(32000)
boy_train_idx = np.random.choice(boy_url_sample, len(boy_url_sample)*training_size//100, replace=False)
boy_train_data = np.array(boy_url_sample[boy_train_idx,:])
girl_train_idx = np.random.choice(girl_url_sample, len(girl_url_sample)*training_size//100, replace=False)
girl_train_data = np.array(girl_url_sample[girl_train_idx,:])
boy_val_idx = np.array([item for item in tqdm(arrint) if item not in boy_train_idx])
boy_val_data = np.array(boy_url_sample[boy_val_idx,:])
girl_val_idx = np.array([item for item in tqdm(arrint) if item not in girl_train_idx])
girl_val_data = np.array(girl_url_sample[girl_val_idx,:])
print("train boys: ", len(boy_train_data))
print("train girls: ", len(girl_train_data))
print("validation boys: ", len(boy_val_data))
print("validation girls: ", len(girl_val_data))

IndexError: arrays used as indices must be of integer (or boolean) type